In [1]:
from readlif.reader import LifFile
import numpy as np
from dask import delayed
import dask.array as da

import zarr
from ome_zarr.io import parse_url
from ome_zarr.writer import write_image

In [3]:
exp_path = r'R:\Dante\hillman_academy_2023\Jinny_test.lif'

In [4]:
# load experiment
exp = LifFile(exp_path)

In [5]:
# get names and dimensions of all elements in the lif file
[[image.name, image.dims] for image in exp.get_iter_image()]

[['G6 Region1', Dims(x=2048, y=2048, z=1, t=1, m=76)],
 ['G6 Region1_Merged', Dims(x=35181, y=7548, z=1, t=1, m=1)],
 ['TileScan 4/B4 Region1', Dims(x=2048, y=2048, z=1, t=1, m=77)],
 ['TileScan 4/C4', Dims(x=2048, y=2048, z=1, t=1, m=77)],
 ['TileScan 4/D6', Dims(x=2048, y=2048, z=1, t=1, m=77)],
 ['TileScan 4/E6', Dims(x=2048, y=2048, z=1, t=1, m=77)],
 ['TileScan 4/F6', Dims(x=2048, y=2048, z=1, t=1, m=77)],
 ['TileScan 4/G6', Dims(x=2048, y=2048, z=1, t=1, m=77)],
 ['TileScan 4/B4 Region1_Merged', Dims(x=16801, y=16798, z=1, t=1, m=1)],
 ['TileScan 4/C4_Merged', Dims(x=16791, y=16793, z=1, t=1, m=1)],
 ['TileScan 4/D6_Merged', Dims(x=16799, y=16800, z=1, t=1, m=1)],
 ['TileScan 4/E6_Merged', Dims(x=16809, y=16796, z=1, t=1, m=1)],
 ['TileScan 4/F6_Merged', Dims(x=16800, y=16794, z=1, t=1, m=1)],
 ['TileScan 4/G6_Merged', Dims(x=16792, y=16783, z=1, t=1, m=1)],
 ['TileScan 1/B4 Region1', Dims(x=2048, y=2048, z=1, t=1, m=77)],
 ['TileScan 1/C4', Dims(x=2048, y=2048, z=1, t=1, m=77)],

In [6]:
# get a list based on criteria regarding dimensions
[image.dims for image in exp.get_iter_image() if ((image.dims.t>100) and (image.dims.x > 2048))]

[Dims(x=16792, y=16800, z=1, t=241, m=1),
 Dims(x=16796, y=16798, z=1, t=241, m=1),
 Dims(x=16794, y=16797, z=1, t=241, m=1),
 Dims(x=16802, y=16793, z=1, t=241, m=1),
 Dims(x=16808, y=16802, z=1, t=241, m=1),
 Dims(x=16796, y=16790, z=1, t=241, m=1)]

In [7]:
# get a a selected series
sel_name = 'TileScan 5/E6_Merged'
t = [image for image in exp.get_iter_image() if sel_name in image.name][0]
t

'LifImage object with dimensions: Dims(x=16802, y=16793, z=1, t=241, m=1)'

In [29]:
from numcodecs import Blosc
compressor = Blosc(cname='lz4', clevel=1, shuffle=Blosc.NOSHUFFLE)

In [30]:
def frame_getter(lif,ind):

    return np.array(lif.get_frame(z=0, t=ind, c=1))

lazy_arrays = [delayed(frame_getter)(t,i) for i in range(t.dims.t)]
dask_arrays = [da.from_delayed(delayed_reader, shape=[16793,16802], dtype='uint16') for delayed_reader in lazy_arrays]
stack = da.stack(dask_arrays, axis=0)

# save zarr file

size_t = 50
size_xy = 128

zarr_path = r'D:\kasia\tracking\zarr_tests\E6_C1_c1.zarr'

# write the image data
store = parse_url(zarr_path, mode="w").store
root = zarr.group(store=store)

# it will fail if the store already contains arrays
write_image(image=stack, group=root, axes="tyx", storage_options=dict(chunks=(size_t,size_xy, size_xy),compressor=compressor))

[]

In [28]:
%time
zarr_array = zarr.open(zarr_path)
zarr_array[0].info

CPU times: total: 0 ns
Wall time: 0 ns


Name,/0
Type,zarr.core.Array
Data type,int32
Shape,"(1, 1000, 1000)"
Chunk shape,"(1, 100, 100)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=NOSHUFFLE, blocksize=0)"
Store type,zarr.storage.DirectoryStore
No. bytes,4000000 (3.8M)
No. bytes stored,395


In [23]:
zarr_array = zarr.open(zarr_path)
zarr_array[0].info

Name,/0
Type,zarr.core.Array
Data type,int32
Shape,"(1, 100, 100)"
Chunk shape,"(1, 10, 10)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=5, shuffle=NOSHUFFLE, blocksize=0)"
Store type,zarr.storage.DirectoryStore
No. bytes,40000 (39.1K)
No. bytes stored,391


In [5]:
from tifffile import imread
import dask.array as da

In [2]:
labels_example_path = r'D:\kasia\tracking\E6_exp\sample_data\track_labels.tif'

In [3]:
labels = imread(labels_example_path)

In [8]:
labels_da = da.from_array(labels, chunks=(10, 128, 128))

In [7]:
labels_da

dask.array<array, shape=(20, 1000, 1000), dtype=int32, chunksize=(20, 128, 128), chunktype=numpy.ndarray>

In [11]:
from numcodecs import Blosc
compressor = Blosc(cname='lz4', clevel=1, shuffle=Blosc.BITSHUFFLE)

# save zarr file

size_t = 10
size_xy = 128

zarr_path = r'D:\kasia\tracking\zarr_tests\labels_c1.zarr'

# write the image data
store = parse_url(zarr_path, mode="w").store
root = zarr.group(store=store)

# it will fail if the store already contains arrays
write_image(image=labels_da, group=root, axes="tyx", storage_options=dict(chunks=(size_t,size_xy, size_xy),compressor=compressor))

[]

In [12]:
zarr_array = zarr.open(zarr_path)
zarr_array[0].info

Name,/0
Type,zarr.core.Array
Data type,int32
Shape,"(20, 1000, 1000)"
Chunk shape,"(10, 128, 128)"
Order,C
Read-only,False
Compressor,"Blosc(cname='lz4', clevel=1, shuffle=BITSHUFFLE, blocksize=0)"
Store type,zarr.storage.DirectoryStore
No. bytes,80000000 (76.3M)
No. bytes stored,397


In [15]:
zarr_path

'D:\\kasia\\tracking\\zarr_tests\\labels_c1.zarr'